In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer, RobertaConfig

# Load preprocessed data
df = pd.read_csv('preprocessed_tweets_dataset.csv')

# Mapping categorical labels to integers
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df['label'] = df['Sentiment'].map(label_map)

assert df['label'].isin([0, 1, 2]).all(), "Labels must be in the range [0, 2]"

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['preprocessed_text'], df['label'], test_size=0.2, random_state=42)

# Convert train_texts and test_texts to lists of strings
train_texts = train_texts.astype(str).tolist()
test_texts = test_texts.astype(str).tolist()

# Initialize RoBERTa tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
config = RobertaConfig.from_pretrained(model_name, num_labels=3)


# Tokenize text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Ensure train_labels and test_labels are integers
train_labels = train_labels.astype(int).values
test_labels = test_labels.astype(int).values

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(len(train_texts)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(16)

# Load pre-trained RoBERTa model for sequence classification
model = TFRobertaForSequenceClassification.from_pretrained(model_name, config=config)

# Define model compile and training
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

# Train the model
history = model.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Evaluate model performance
predictions = model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=1)
print(classification_report(test_labels, predicted_labels.numpy(), target_names=['Negative', 'Neutral', 'Positive']))

# Save model training and evaluation information in documentation file (README.md)
model_training_info = """
### Model Training and Evaluation

- **Model:** Fine-tuned BERT for Sequence Classification
- **Metrics:** Accuracy, Precision, Recall, F1 Score
"""
with open('README.md', 'a') as readme_file:
    readme_file.write(model_training_info)

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceC

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
244/244 [==============================] - 927s 4s/step - loss: 0.9730 - accuracy: 0.4987 - val_loss: 0.8056 - val_accuracy: 0.6232
Epoch 2/3
244/244 [==============================] - 857s 4s/step - loss: 0.7425 - accuracy: 0.6781 - val_loss: 0.7227 - val_accuracy: 0.6704
Epoch 3/3
61/61 [==============================] - 56s 829ms/step
              precision    recall  f1-score   support

    Negative       0.61      0.72      0.66       269
     Neutral       0.73      0.52      0.61       378
    Positive       0.69      0.82      0.75       327

    accuracy                           0.67       974
   macro avg       0.68      0.69      0.67       974
weighted avg       0.68      0.67      0.67       974

